# To do

* extract and print residual forces from geom optimisation etc etc 

In [ ]:
from ase.io import read
import sys
import glob
import re

sys.path.append("/home/b55k/harryrich11.b55k/INS")
from inelasty_data import inelasty

In [3]:
vasp_file_path = "/home/b55k/harryrich11.b55k/vasp"
conda_command = "source ~/miniforge3/bin/activate vasp"
spack_command = """. ~/spack/share/spack/setup-env.sh 
spack env activate vasp_fin"""

In [4]:
from settings import (
    kpoint_kwargs,
    encut_kwargs,
    geom_relax_kwargs1,
    geom_relax_kwargs2,
    phonopy_kwargs,
)

In [5]:
benz_cryst = "/home/b55k/harryrich11.b55k/benz_cryst_vasp_prod/benzene_solid.cif"
atoms = read(benz_cryst)
benz = inelasty(
    struc=atoms,
    vasp_path=vasp_file_path,
    conda_command=conda_command,
    spack_command=spack_command,
)

/home/b55k/harryrich11.b55k/INS


In [ ]:
benz.relax(
    vasp_kwargs1=geom_relax_kwargs1, vasp_kwargs2=geom_relax_kwargs2, time="08:00:00"
)

In [ ]:
relaxed_struc = (
    "/home/b55k/harryrich11.b55k/benz_cryst_vasp_prod/geom_opt/stage_2/CONTCAR"
)
atoms = read(relaxed_struc)
benz = inelasty(
    struc=atoms,
    vasp_path=vasp_file_path,
    conda_command=conda_command,
    spack_command=spack_command,
)

In [ ]:
kpoints_list = [(1, 1, 1), (2, 2, 2)]
benz.run_k_points(kpoint_kwargs=kpoint_kwargs, kpoints_list=kpoints_list)

In [ ]:
benz.run_encut(vasp_kwargs=encut_kwargs)

In [ ]:
benz.generate_phonopy(
    atom_path=relaxed_struc,
    vasp_kwargs=phonopy_kwargs,
    run=True,
    supercell_size=1,
    ntasks_per_node=24,
    time="24:00:00",
)

In [ ]:
import phonopy
from phonopy.interface.vasp import parse_set_of_forces

In [ ]:
phonon = phonopy.load("phonopy/phonopy_disp.yaml")
outcar_files = sorted(glob.glob("./phonopy/phonopy-*/vasprun.xml"))
# shamelessly stolen from stack overflow sorting regex
outcar_files.sort(
    key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r"(\d+)", x)]
)

sets_of_forces = parse_set_of_forces((12 * 4 * 2**3), outcar_files)
phonon.forces = sets_of_forces["forces"]
phonon.produce_force_constants()
phonon.save(settings={"force_constants": True})